#  Secondary Data
The objective is to experience with working with secondary data.  This notebook defaults to Python 3, as R is ill-suited to string manipulaiton, which is especially important for Task 6.  Some cells can be run in R, however, as shown in Task 5.


## Filtering to find competitors
Write a function `filter_products_raw` that takes one argument (`key_phrase`) and returns a list of strings; these strings should be all the lines in the `/data/Assignment-2/products.json` file which contain the specified key phrase.

In [1]:
import json
import demjson
f = open("/data/Assignment-2/products.json",'r',encoding='UTF-8')
search_list=f.readlines()

In [2]:
def filter_products_raw(key_phrase):
    resultlist=[]
    with open('/data/Assignment-2/products.json', 'r') as reader:
  # Read and print the entire file line by line
        for line in reader:
            line=str.lower(line)
            if line.find(key_phrase) !=-1:
                resultlist.append(line)
        return resultlist
        

In [14]:
filter_products_raw('smartpen')

['{\'asin\': \'b001aaozhi\', \'categories\': [[\'electronics\', \'portable audio & video\', \'digital voice recorders\']], \'description\': \'from the manufacturercapture everythingthe pulse smartpen records audio and links it to what you write. missed something? tap on your notesno need to lug the laptopthe pulse smartpen automatically captures everything as you write and draw. transfer your notes to your computer, organize them, and even search for words within your notes. find what you want in seconds.share your notestransform your notes and audio into interactive movies. upload your creations online for everyone to see, hear and play.what you getthe pulse smartpen (1.3 ounces, anodized aluminum housing)1 gb of memory (actual user available memory will be less)livescribe desktop software (download)3-d recording headset100 sheet dot paper college-ruled notebookusb mobile charging cradlesmartpen case2 black fine point ink cartridgesdemo card and interactive stickersinteractive getting

**Task 2:** Using the function you just wrote, try filtering the products in `products.json` by different key words or phrases.  Print out the total number of products that match the key phrase.  Also print out the first three products that match the key phrase.  Use this technique to find a collection of products that would be in competition with the product your team is exploring.

In [3]:
len(filter_products_raw('smartpen'))

33

**Task 3:** With the `filter_products_raw` function as a starting point, create a new function `filter_products_df` that returns a pandas data frame with two columns: `product.id` and `price`.  

*Hints:*

*1) The product id isn't called that; you need to figure out what it's called in the documentation.*

*2) The json data files aren't proper json for multiple reasons; consider using the `demjson` library.*

*3) You may need to ignore products for which prices are missing.*

In [4]:
import ast
import demjson
import pandas as pd

def filter_products_df(key_phrase):
    product_id_list=[]
    price_list=[]
    for i in filter_products_raw(key_phrase):
        mid_dic = demjson.decode(i)
        try: 
            price_list.append(mid_dic['price'])
            product_id_list.append(mid_dic['asin'])
        except:
            continue 
        #i = ast.literal_eval(i)
        #idlist.append(i.get('asin'))
        #pricelist.append(i.get('price'))
    df = pd.DataFrame(list(zip(product_id_list,price_list)),columns = ['product.id','price'])
    df.dropna(inplace=True)
    return df

**Task 4:** Using the new `filter_products_df` function, filter the products by the key phrase you identified in Task 2, resulting in a data frame of product ids with their corresponding prices; call this data frame variable `products`.

In [16]:
products = filter_products_df("smartpen")
products
products.to_csv("product.csv",index=False,sep=',')

**Task 5:** Plot the distribution of prices for products matching your query using the code provided in the following three cells.

In [6]:
# Set up R
import tzlocal
import rpy2.ipython
%reload_ext rpy2.ipython

In [7]:
%%R -i products
library(ggplot2)
ggplot(products, aes(x=price)) + geom_histogram() + theme_bw()

C:\App\Aconda\lib\site-packages\rpy2\robjects\pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\szdxl\\AppData\\Local\\Temp\\tmpk0z77l4n\\Rplots001.png'

In [27]:
%%R
ggsave("price_histogram.pdf", width=6.5, height=3)


Error in ggsave("price_histogram.pdf", width = 6.5, height = 3) : 
  could not find function "ggsave"


PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\szdxl\\AppData\\Local\\Temp\\tmp_js6k0j0\\Rplots001.png'

## Filtering competitor reviews
**Task 6:** Write a function called `filter_reviews` to help you find all the reviews that match the products which match your key phrase query.  This function should take a `products` argument and return a data frame with one review per row and three columns: `product.id`, `rating` (on the 5-star scale), and `review` (text).

*Hints:*

*1) There are lots of really inefficient ways to do this.  If running this function takes more than two minutes, you're doing something wrong.*

*2) I recommend testing things out on data subsamples so that you're sure of the efficiency of your code.  (For example, you can return after the first matching review).*

*3) Decoding the json on each line isn't really necessary; try doing it only for the reviews that match your product list.*

*4) String manipulation with `split` is powerful.*

In [14]:
# %load /data/Assignment-2/Task6.py
from demjson import decode
def filter_reviews(products):
    df = pd.DataFrame({"product.id": [], "rating": [], "review": []})
    f = open("reviews.json")
    prods = set(products["product.id"])
    for line in f:
        line=str.lower(line)
        id = line.split(':')[2].split(',')[0].strip().strip('"')
        if id in prods:
            review = decode(line.strip())
            d = pd.DataFrame({"product.id": [review['asin']], \
                              "rating": [review['overall']], \
                              "review": [review['reviewtext']]})
            df = df.append(d, ignore_index=True)
    return df


Now filter reviews by the produts of interest found in Task 4.

In [17]:
filter_reviews(products)
filter_reviews(products).to_csv("reviews.csv",index=False,sep=',')

**Task 7:** Summarize the reviews into a data frame called `review_summaries` with three columns: `product.id`, `count` for the number of reviews, and `ave.rating` for the mean rating.

In [40]:
review_summaries = filter_reviews(products).groupby('product.id')\
                    .agg({'review':'count','rating':'mean'})\
                    .rename(columns={'review':'count','rating':'avg.rating'})\
                    .reset_index()
print(review_summaries)

    product.id  count  avg.rating
0   b001aan4pw    146    3.890411
1   b001aaozhi     88    4.238636
2   b001qr6fn2      1    2.000000
3   b001r4i74e     63    4.031746
4   b001r4kb3e      2    4.500000
5   b001r4kb66      3    4.666667
6   b002djtmsw     68    3.926471
7   b002djv83y    101    3.732673
8   b0039mzbfk      6    4.000000
9   b003rae19q    637    3.444270
10  b003rapakk     74    3.513514
11  b003to56f8     68    3.617647
12  b003to56fi     38    3.526316
13  b003unl14s     12    4.250000
14  b004mf1e7m      2    2.000000
15  b004wkkrts      6    4.833333
16  b0050affwi     37    4.162162
17  b0053uemho      1    5.000000
18  b0055uijhg      1    5.000000
19  b005a0b6ks      9    3.222222
20  b007jus330     27    4.074074
21  b008ysguv8      4    4.000000
22  b008zxxx2q     47    2.659574
23  b00cgtlovw      1    1.000000
24  b00cvtl3bs      1    1.000000
25  b00cx49v38      1    1.000000
26  b00fg38l16    214    3.200935
27  b00gkinxtg      5    3.600000
28  b00gkinxuu

**Task 8:** Merge `review_summaries` with `products` on the `product.id` variable.

In [41]:
review_summaries = review_summaries.merge(products,left_on='product.id', right_on='product.id')
review_summaries

,product.id,count,avg.rating,price
0,b001aan4pw,146,3.890411,289.99
1,b001aaozhi,88,4.238636,289.99
2,b001qr6fn2,1,2.000000,117.57
3,b001r4i74e,63,4.031746,5.76
4,b001r4kb3e,2,4.500000,17.06
5,b001r4kb66,3,4.666667,17.06
6,b002djtmsw,68,3.926471,117.57
7,b002djv83y,101,3.732673,289.99
8,b0039mzbfk,6,4.000000,117.57
9,b003rae19q,637,3.444270,4.47


In [43]:
review_summaries.to_csv('summary.csv', index=False, sep=',')

**Task 9:** Using `ggplot`, Plot a scatterplot of the review summaries with price on the x-axis, the average rating on the y axis, and the size of each point being the number of reviews.  Add labels for the product ids, but fix the label size to be the same for all products.

*Hints:*

*1) You'll want to start the cell with `%%R -i review_summaries`.*

*2) You may want to use `scale_size_continuous(range = c(0.1,10)) + geom_text(size=1)` or similar.*

In [51]:
%%R -i review_summaries
ggplot(data=review_summaries,aes(x=price,y=avg.rating,label=product.id))+geom_point(aes(size = count))+scale_size_continuous(range = c(0.1,10)) + geom_text(size=3,color='red') 



Error in ggplot(data = review_summaries, aes(x = price, y = avg.rating,  : 
  could not find function "ggplot"


C:\App\Aconda\lib\site-packages\rpy2\robjects\pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\szdxl\\AppData\\Local\\Temp\\tmpapp3jrcr\\Rplots001.png'

In [82]:
%%R
ggsave("reviews_summary.pdf", width=6.5, height=4)

**Task 10:** Write function `filter_products_by_id` that takes in a product identifier argument `idx` and returns returns the matching product information from `/data/Assignment-2/products.json`.

In [44]:
ast.literal_eval(search_list[1]).get('price')

In [45]:
def filter_products_by_id(idx):
    product_name=''
    product_price=''
    for i in search_list:
        if idx == ast.literal_eval(i).get('asin'):
            product_name = ast.literal_eval(i).get('title')
            product_price = ast.literal_eval(i).get('price')
    df=pd.DataFrame({'product.name':[product_name],'product.id':[idx],'price':[product_price]})            
    return df

In [49]:
filter_products_by_id('B00FG38L16')


,product.name,product.id,price
0,Livescribe 3 Smartpen for iOS7 iPhone and iPad,B00FG38L16,149.95


**Task 11:** Using the plot form Task 9, the `filter_products_by_id` function, and the `review_summaries` data frame, identify 3 competitor products and extract the relevant information required for the assignment write-up.

In [ ]:
products=filter_products_by_id('B00FG38L16')
review_summaries = filter_reviews(products).groupby('product.id')\
                    .agg({'review':'count','rating':'mean'})\
                    .rename(columns={'review':'count','rating':'avg.rating'})\
                    .reset_index()
review_summaries = review_summaries.merge(products,left_on='product.id', right_on='product.id')
review_summaries

**Task 12:** Filter the `reviews` data frame for each of the products so that you may read the corresponding reviews.

*Hint: You may wish to save the filtered sets of reviews to separate files with the `to_csv` using the argument `sep='\t'`.  Then you can download these file to make the reviews easier to read.  You can also read the reviews right here, too!*

In [121]:
filter_reviews(products).to_csv(index=False,sep='\t')

"product.id\trating\treview\nB00AW62TSS\t1.0\tThis does not work on my Hoveround Teknique.  So this ad must be misleading. It plugs in, but it doesn't charge the chair.  I was in the hospital when it came so we had it for over a month before I could try it.  Haven't sent it back but I am going to see if they will refund me.\n"